In [2]:
import sys
sys.path.append("..")

In [3]:
from dtcontrol.dataset.dataset_loader import DatasetLoader
from dtcontrol.dataset.scots_dataset_loader import ScotsDatasetLoader
import numpy as np

In [7]:
controller = 'examples/10rooms.scs'
X, _, y, _, index_to_value = ScotsDatasetLoader().load_dataset(controller)

  9%|▊         | 2265/26244 [00:00<00:02, 11025.81it/s]

Reading from examples/10rooms.scs


 97%|█████████▋| 25504/26244 [00:02<00:00, 11450.90it/s]

Constructed training set with 26244 datapoints


In [8]:
csv = controller.replace('scs', 'csv')

f = open(csv, 'w')
for i in range(X.shape[0]):
    if len(y.shape) == 2:
        for e in y[i]:
            if e == -1:
                break
            f.write(",".join(map(str, X[i]))+","+str(index_to_value[e])+"\n")
    else:
        stacked_y = np.stack(y, axis=2)
        for e in stacked_y[i]:
            if e[0] != -1:
                f.write(",".join(map(str, X[i]))+","+",".join(map(str, map(lambda x: index_to_value[x], e)))+"\n")
f.close()

In [27]:
import pandas as pd

csv = 'examples/10rooms.csv'

f = open(csv, 'r')
state_dim, input_dim = map(int, f.readline().split("BEGIN")[1].split())

ds = pd.read_csv(f, header=None)

unique_list = []
for i in range(state_dim, state_dim+input_dim):
    unique_list += ds[i].unique().tolist()
index_to_value = {x: y for x, y in enumerate(set(unique_list))}
value_to_index = {y: x for x, y in index_to_value.items()}

ds[[i for i in range(state_dim, state_dim+input_dim)]] = ds[[i for i in range(state_dim, state_dim+input_dim)]].applymap(lambda x: value_to_index[x])

grouped = ds.groupby([i for i in range(state_dim)], sort=False)
aggregate = grouped[state_dim].apply(list).reset_index(name=state_dim)
for i in range(1, input_dim):
    aggregate[state_dim+i] = grouped[state_dim+i].apply(list).reset_index(name=state_dim+i)[state_dim+i]

max_non_det = aggregate[state_dim].agg(len).max()

for i in range(0, input_dim):
    aggregate[state_dim+i] = aggregate[state_dim+i].apply(lambda ls: ls + [-1 for i in range(max_non_det - len(ls))])

X_train = np.array(aggregate[[i for i in range(state_dim)]])

if input_dim > 1:
    Y_train = np.ndarray((input_dim, X_train.shape[0], max_non_det), dtype=np.int16)
    for i in range(input_dim):
        Y_train[i] = np.array(aggregate[state_dim+i].tolist())
else:  # input_dim = 1
    Y_train = np.array(aggregate[state_dim].tolist())

In [29]:
Y_train

array([[[ 1, -1, -1, ..., -1, -1, -1],
        [ 1, -1, -1, ..., -1, -1, -1],
        [ 1, -1, -1, ..., -1, -1, -1],
        ...,
        [ 0,  2, -1, ..., -1, -1, -1],
        [ 0, -1, -1, ..., -1, -1, -1],
        [ 0, -1, -1, ..., -1, -1, -1]],

       [[ 1, -1, -1, ..., -1, -1, -1],
        [ 1, -1, -1, ..., -1, -1, -1],
        [ 1, -1, -1, ..., -1, -1, -1],
        ...,
        [ 0,  0, -1, ..., -1, -1, -1],
        [ 0, -1, -1, ..., -1, -1, -1],
        [ 0, -1, -1, ..., -1, -1, -1]]], dtype=int16)